In [1]:
import json
import os
import autogen
from autogen import ConversableAgent, GroupChat, GroupChatManager
import pandas as pd
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from langchain_community.vectorstores import chroma

In [2]:
config_list = [{"model": "mistral","api_type":"ollama"}]
llm_config={
    "timeout":600,
    "cache_seed":42,
    "temperature":0.1,
    "config_list":config_list,}

In [3]:
def csv_to_json(csv_path):
    df=pd.read_csv(csv_path)
    normalized_logs = df.to_dict(orient="records")
    with open("normalized_logs.json", "w") as f:
        json.dump(normalized_logs, f, indent=4)
    return normalized_logs
    
cyber_logs=csv_to_json("Mock cybersecurity logs.csv")
threat_policy=csv_to_json("Mock threat policy definitions.csv")

In [4]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE"==str(x.get("content", ""))[-9:].upper()

In [5]:
anomaly_detector = RetrieveUserProxyAgent(
    name = "anomaly_detector",
    system_message = '''You are cybersecurity agent. You have an extra retrieval super power. 
    You retrieve data from cyber_logs and detect anomalies and suspicious pattern using llm intelligence.
    List all the anomalies and suspicious pattern you have detected.''',
    llm_config = llm_config,
    code_execution_config = False,
    max_consecutive_auto_reply = 3,
    retrieve_config = {
        "task": "qa",
        "docs_path": ["Mock cybersecurity logs.csv"],
        "docs_content": cyber_logs,
        "chunk_token_size": 100,
        "model": config_list[0]["model"],
        "vector_db": "chromadb",
        "get_or_create": True,},
    is_termination_msg = termination_msg,
    human_input_mode="NEVER",
    default_auto_reply="Return 'TERMINATE' when you have gathered all the information."
)

threat_classifier_agent = ConversableAgent(
    name = "threat_classifier_agent",
    system_message = '''You are cybersecurity agent. You gather anomalies and suspicious pattern from anomaly_detector
    and classify the threat level 
    (e.g., Compromised Admin: Critical, 
    IoT Anomaly: Medium, 
    Unauthorized changes to backup config: High).''',
    llm_config = llm_config,
    code_execution_config = False,
    max_consecutive_auto_reply = 3,
    is_termination_msg = termination_msg,
    human_input_mode="NEVER",
    default_auto_reply="Return 'TERMINATE' when you have gathered all the information."
)

validator_and_responder_agent = RetrieveUserProxyAgent(
    name = "validator_and_responder_agent",
    system_message = '''You are cybersecurity agent. You have an extra retrieval super power. 
    You gather data from anomaly_detector and threat_classifier_agent. 
    You validate the threat level classified by threat_classifier_agent using internal "threat_policy" document.
    Based on the internal threat_policy document, suggests response actions 
    (e.g., Malware Detection: Critical - response: Isolate infected host immediately,
    Unauthorized changes to backup config: High - response: Alert backup admin)''',
    llm_config = llm_config,
    code_execution_config = False,
    max_consecutive_auto_reply = 3,
    retrieve_config = {
        "task": "qa",
        "docs_path": ["Mock cybersecurity logs.csv"],
        "docs_content": threat_policy,
        "chunk_token_size": 100,
        "model": config_list[0]["model"],
        "vector_db": "chromadb",
        "get_or_create": True,},
    is_termination_msg = termination_msg,
    human_input_mode="NEVER",
    default_auto_reply="Return 'TERMINATE' when you have gathered all the information."
)

In [6]:
groupchat = GroupChat(
    agents=[anomaly_detector, threat_classifier_agent, validator_and_responder_agent],
    messages=[],
    max_round=3,
    speaker_selection_method="round_robin"
)

In [7]:
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)
manager.initiate_chat(
    anomaly_detector,
    message="Detect if there is any anomaly and suspicious pattern from the log."
)


chat_manager (to anomaly_detector):

Detect if there is any anomaly and suspicious pattern from the log.

--------------------------------------------------------------------------------
anomaly_detector (to chat_manager):

1. Unusual number of failed login attempts within a short timeframe from an unknown IP address: This could indicate a brute force attack attempt.

2. Multiple unauthorized access attempts from the same IP address in a short period: This suggests that an unauthorized user is attempting to gain access to sensitive systems or data.

3. Abnormal amount of data transfer to external servers, especially outside of usual business hours: Large-scale data transfers, particularly during non-business hours, could indicate potential data theft or exfiltration.

4. Multiple failed attempts to download or upload large files: Unusual file transfer activities may signal an attempt to introduce malware or steal sensitive information.

5. Inbound and outbound traffic on non-standard p

ChatResult(chat_id=None, chat_history=[{'content': 'Detect if there is any anomaly and suspicious pattern from the log.', 'role': 'assistant', 'name': 'chat_manager'}, {'content': '1. Unusual number of failed login attempts within a short timeframe from an unknown IP address: This could indicate a brute force attack attempt.\n\n2. Multiple unauthorized access attempts from the same IP address in a short period: This suggests that an unauthorized user is attempting to gain access to sensitive systems or data.\n\n3. Abnormal amount of data transfer to external servers, especially outside of usual business hours: Large-scale data transfers, particularly during non-business hours, could indicate potential data theft or exfiltration.\n\n4. Multiple failed attempts to download or upload large files: Unusual file transfer activities may signal an attempt to introduce malware or steal sensitive information.\n\n5. Inbound and outbound traffic on non-standard ports: Traffic flowing through uncom